In [1]:
from utils import *

h2 = H2Toolkit()
h2.check_torch_gpu()

Torch version: 2.0.0+cu117
Torch build with CUDA? True
# Device(s) available: 1, Name(s): NVIDIA GeForce RTX 3080


In [2]:
all_data = h2.read_data(n_subsample=1000)
X_dataset, y_dataset, x_scaler, y_scaler = h2.process_data(restype='SA')

CH4: (10000, 17) | CO2: (10000, 17) | N2: (10000, 17) | NOCG: (10000, 17)
All: (1000, 17)
Full dataset shape: (1000, 17)
Truncated dataset shape: (1000, 17)
Clean (no outliers) dataset shape: (917, 17)
Dataset: X=(917, 12) | y=(917, 3)


In [3]:
X_train, X_valid, X_test, y_train, y_valid, y_test = h2.train_valid_test_split()

X_train_tensor, X_valid_tensor, X_test_tensor, y_train_tensor, y_valid_tensor, y_test_tensor = h2.arrays_to_tensors()

train_dataset, train_loader, valid_dataset, valid_loader= h2.make_dataloader()

X: train: (624, 12) | validation: (109, 12) | test: (184, 12)
y: train: (624, 3)  | validation: (109, 3)  | test: (184, 3)


In [4]:
rom = h2_cushion_rom(X_dataset.shape[-1], y_dataset.shape[-1], [32,64,128,64,32,16])

optimizer = optim.Adam(rom.parameters(), lr=1e-3)
loss_fn   = customLoss()

dict_log = h2.train(rom, loss_fn, optimizer, 100, train_loader, valid_loader)

  0%|          | 0/100 [00:01<?, ?it/s]


TypeError: 'int' object is not iterable

In [ ]:
plt.plot(dict_log['loss'], label='loss')
plt.plot(dict_log['val_loss'], label='validation loss')
plt.legend()
plt.show()

In [ ]:
y_train_pred = np.array(rom(X_train_tensor).tolist())
y_test_pred  = np.array(rom(X_test_tensor).tolist())
print('Train Pred shape: {}'.format(y_train_pred.shape))
print('Test Pred shape:  {}'.format(y_test_pred.shape))

In [ ]:
mse_train = mean_squared_error(y_train, y_train_pred)
r2_train  = r2_score(y_train, y_train_pred)
print('Train -- MSE={:.2e} | R^2={:.2f}'.format(mse_train, r2_train))

mse_test = mean_squared_error(y_test, y_test_pred)
r2_test  = r2_score(y_test, y_test_pred)
print('Test  -- MSE={:.2e} | R^2={:.2f}'.format(mse_test, r2_test))

In [ ]:
plt.figure(figsize=(15,3))
for i in range(3):
    r2train = r2_score(y_train[:,i], y_train_pred[:,i])
    r2test  = r2_score(y_test[:,i],  y_test_pred[:,i])
    plt.subplot(1,3,i+1)
    plt.scatter(y_train[:,i], y_train_pred[:,i], alpha=0.5, label='train')
    plt.scatter(y_test[:,i],  y_test_pred[:,i], alpha=0.5, label='test')
    plt.axline([0,0],[1,1], c='r', linewidth=3)
    plt.xlabel('True'); plt.ylabel('Predicted')
    plt.xlim([-0.1,1.1]); plt.ylim([-0.1,1.1])
    plt.title('$R^2_{}$={:.2f} ; $R^2_{}$={:.2f}'.format('{train}',r2train,'{test}',r2test))
    plt.legend()
plt.show()

In [ ]:
#make_dot(rom(X_tensor), params=dict(rom.named_parameters()))

***
# END